In [39]:
import torch 
from utils import intersection_over_union
from torch import nn

In [2]:
target = torch.rand(5,7,7,4)
prediction = torch.rand(5,7,7,4)

In [5]:
torch.flatten(target,end_dim=-2).shape,torch.flatten(prediction ,end_dim=-2).shape

(torch.Size([245, 4]), torch.Size([245, 4]))

In [70]:
mse = torch.nn.MSELoss(reduction = 'sum')

In [15]:
loss = mse(torch.flatten(target,end_dim=-2),torch.flatten(prediction ,end_dim=-2))

In [67]:
iou_box1 = intersection_over_union(prediction[...,21:25],target[...,21:25])
iou_box2 = intersection_over_union(prediction[...,26:30],target[...,21:25])

In [68]:
iou_box2.shape ,iou_box1.shape

(torch.Size([5, 7, 7, 1]), torch.Size([5, 7, 7, 1]))

In [73]:
ious = torch.cat([iou_box2.unsqueeze(0) ,iou_box1.unsqueeze(0)],dim=0)

In [74]:
ious.shape

torch.Size([2, 5, 7, 7, 1])

In [75]:
#best_box 0 if the iou_box1 is biger than iou_box2 else 1
iou_max ,best_box = torch.max(ious,dim=0)

In [76]:
iou_max.shape ,best_box.shape

(torch.Size([5, 7, 7, 1]), torch.Size([5, 7, 7, 1]))

In [77]:
exist_box = target[...,20].unsqueeze(3)
exist_box.shape

torch.Size([5, 7, 7, 1])

In [ ]:
iou_max,best_box

In [43]:
torch.tensor([[[1],[2]],[[4],[5]],[[7],[8]]]).shape

torch.Size([3, 2, 1])

In [87]:
target = torch.rand(5,7,7,25)
prediction = torch.rand(5,7,7,30)

In [83]:
mse = nn.MSELoss(reduction="sum")

In [88]:
loss  = forward(prediction , target)

In [18]:
import torch

In [19]:
from utils import (
convert_cellboxes,
cellboxes_to_boxes
)

In [28]:
pred = torch.rand(16,588)
cover_pred = convert_cellboxes(pred)

In [54]:
converted_pred = convert_cellboxes(pred).reshape(pred.shape[0], 7 * 7, -1)
converted_pred[..., 0] = converted_pred[..., 0].long()
all_bboxes = []

for ex_idx in range(pred.shape[0]):
    bboxes = []

    for bbox_idx in range(7 * 7):
        bboxes.append([x.item() for x in converted_pred[ex_idx, bbox_idx, :]])
    all_bboxes.append(bboxes)
print(len(all_bboxes[0]))

49


In [46]:
all_bboxes = cellboxes_to_boxes(pred)

In [49]:
len(all_bboxes)

16

In [33]:
cover_pred = cover_pred.reshape(16, 7 * 7, -1)

In [34]:
cover_pred.shape

torch.Size([16, 49, 6])

In [35]:
cover_pred[..., 0] = cover_pred[..., 0].long()
cover_pred.shape

torch.Size([16, 49, 6])

In [9]:
pred = pred.reshape(16,7,7,30)

In [15]:
predicted_class = pred[..., :20].argmax(-1).unsqueeze(-1)

In [17]:
predicted_class.shape

torch.Size([16, 7, 7, 1])

In [1]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch import optim
from model import Yolov1
from loss import YoloLoss
from dataset import Data_YOLO
from utils import (
    nms,
    cellboxes_to_boxes,
    get_bboxes,
    convert_truelabel_cellboxes
)
from tqdm import tqdm

In [37]:
DEVICE = "cuda" if torch.cuda.is_available else "cpu"
LEARNING_RATE = 2e-5
BATCH_SIZE = 16 # 64 in original paper but I don't have that much vram, grad accum?
WEIGHT_DECAY = 0
EPOCHS = 100
NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = False
LOAD_MODEL_FILE = "overfit.pth.tar"
IMG_DIR = "C:\\Users\\LastHour\\Desktop\\ML_folder\\Computer vsion\YOLO\yolov5\\floss detection\\20DH-2DH-data_augmentaion\\images"
LABEL_DIR = "C:\\Users\\LastHour\\Desktop\\ML_folder\\Computer vsion\YOLO\yolov5\\floss detection\\20DH-2DH-data_augmentaion\\labels"

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(256, interpolation=3),
    transforms.CenterCrop(224),
    
])

In [38]:
data_train = Data_YOLO(LABEL_DIR,IMG_DIR,transform = transform)
data_train_loader = DataLoader(data_train, batch_size = 16 , shuffle=True)


model = Yolov1(split_size=7, num_boxes=2, num_classes=2)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
loss_fn = YoloLoss()

In [55]:
def train_fn(train_loader, model, optimizer, loss_fn):
    loop = tqdm(train_loader, leave=True)
    mean_loss = []

    for batch_idx, (x, y) in enumerate(loop):
        x, y = x , y
        print(y.shape)

        out = model(x)
        loss = loss_fn(out, y)
        mean_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # update progress bar
        out = convert_cellboxes(out)
        loop.set_postfix(loss=loss.item(),out_shape=out.shape)
        
        break

    print(f"Mean loss was {sum(mean_loss)/len(mean_loss)},{out.shape}")

    return out

In [ ]:
for i in range(5):
    train_fn(data_train_loader , model , optimizer , loss_fn)
    
    
    
    break

In [65]:
def convert_truelabel_cellboxes(y , S=7):
    y_class = y[...,0:2].argmax(-1).unsqueeze(-1)
    covert_y = torch.cat(y[...,2:] , y_class , dim=-1)

    return covert_y

In [12]:
y = torch.rand(16,7,7,7)

def convert_truelabel_cellboxes(y , S=7):
    #convert class :
    y_class = y[...,0:2].argmax(-1).unsqueeze(-1)

    #convert coordes :
    cell_indices = torch.arange(7).repeat(16, 7, 1).unsqueeze(-1)
    x_coord = 1 / S * (y[..., 3:4] + cell_indices)
    y_coord = 1 / S * (y[..., 4:5] + cell_indices.permute(0, 2, 1, 3))
    w_y = 1 / S * y[..., 5:7]
    #covert score objectenss:
    score_objtenss = y[...,2:3]

    covert_y = torch.cat((y_class,score_objtenss,x_coord,y_coord,w_y) , dim=-1)

    return covert_y

In [13]:
def cellboxes_Y_True_to_boxes(Y_True, S=7):
    converted_Y_True = convert_truelabel_cellboxes(Y_True).reshape(Y_True.shape[0], S * S, -1)
    converted_Y_True[..., 0] = converted_Y_True[..., 0].long()
    all_bboxes = []

    for ex_idx in range(Y_True.shape[0]):
        bboxes = []

        for bbox_idx in range(S * S):
            bboxes.append([x.item() for x in converted_Y_True[ex_idx, bbox_idx, :]])
        all_bboxes.append(bboxes)

    return all_bboxes

In [15]:
all_bboxes = cellboxes_Y_True_to_boxes(y)

In [17]:
len(all_bboxes[0])

49

In [6]:
def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):

    """
    Calculates intersection over union

    Parameters:
        boxes_preds (tensor): Predictions of Bounding Boxes (BATCH_SIZE, 4)
        boxes_labels (tensor): Correct labels of Bounding Boxes (BATCH_SIZE, 4)
        box_format (str): midpoint/corners, if boxes (x,y,w,h) or (x1,y1,x2,y2)

    Returns:
        tensor: Intersection over union for all examples
    """

    if box_format == "midpoint":
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2

    if box_format == "corners":
        box1_x1 = boxes_preds[..., 0:1]
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4]  # (N, 1)
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    # .clamp(0) is for the case when they do not intersect
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)

    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    return intersection / (box1_area + box2_area - intersection + 1e-6)


def nms(bboxes , objectenss_proba , iou_threshold ,box_format="midpoint"):
    '''
        bboxes (list): list of lists containing all bboxes with each bboxes
        specified as [[class_pred, prob_score, x1, y1, x2, y2]]
        iou_threshold (float): threshold where predicted bboxes is correct
        threshold (float): threshold to remove predicted bboxes (independent of IoU) 
        box_format (str): "midpoint" or "corners" used to specify bboxes
    Returns:
        list: bboxes after performing NMS given a specific IoU threshold
    '''
    bboxes = [box for box in bboxes if box[1] > objectenss_proba]
    bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)
    NMS_box = list()

    while bboxes :
        chosen_box = bboxes.pop(0)

        bboxes = [
            box
            for box in bboxes
            if box[0] != chosen_box[0] # this line to keep up the other class boxes
            or intersection_over_union(
                torch.tensor(chosen_box[2:]),
                torch.tensor(box[2:]),
                box_format=box_format,
            )
            > iou_threshold
        ]

        NMS_box.append(chosen_box)

    return NMS_box

In [5]:
import torch

In [11]:
boxes = [
    ["car", 0.92, float(100), float(50), float(200),float(150)],
    ["car", 0.85, float(300), float(200), float(400), float(400)],
    ["dog", 0.76, float(50), float(100), float(150), float(250)],
    ["cat", 0.68, float(200), float(300), float(300), float(450)]
]

boxes = nms(boxes , 0.5 , 0.5)

In [12]:
boxes

[['car', 0.92, 100.0, 50.0, 200.0, 150.0],
 ['dog', 0.76, 50.0, 100.0, 150.0, 250.0],
 ['cat', 0.68, 200.0, 300.0, 300.0, 450.0]]